# query_printout.ipynb

## This allows a user to make a query and print out results

In [1]:
import csv
import os
import re
import shutil

import numpy as np
import sqlalchemy as sql

# from . import utils #(in script)
# from . import orm #(in script)

### User input variables

In [18]:
user_input_fields = ["cell_type"]
user_input_filters = {"organism": ["= \"H. sapiens\""],
                      "fcgene_date": ["not null"],
                      "baseline_control_expt": ["in (\"control\",\"baseline\")"]}
#                      "cell_type": ["in (\"ESC\",\"HeLa\")"]}
outfile = "/home/lsanford/Desktop/db_query_human_control_baseline.csv"

In [19]:
# %load utils.py
# utils.py --- Utilities for simplifying database code
#
# Filename: utils.py
# Description: Miscellaneous utilities for simplifying database code
# Author: Zachary Maas <zama8258@colorado.edu> and Lynn Sanford
# Maintainer: Lynn Sanford <lynn.sanford@colorado.edu>
# Created: Mon Jul  1 16:04:05 2019 (-0600)
#

# Commentary:
#
# This module contains a few helpful utility functions and classes for
# reducing the total amount of code needed for the database, since
# there are many areas where the same patterns keep popping up.
#

# Code:

import os
import configparser
import sqlalchemy as sql
import csv
from sqlalchemy.orm import sessionmaker


# Database Connection Handler
class NascentDBConnection:
    engine = None
    _Session = None
    session = None

    def __init__(self, db_url):
        self.engine = sql.create_engine(db_url, echo=False)
        self.Session = sessionmaker(bind=self.engine)
        self.session = self.Session()

    def __enter__(self):
        return self.session

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.commit()
        self.engine.dispose()


# Configuration File Reader
def load_config(filename: str):
    if not os.path.exists(filename):
        raise FileNotFoundError(
            "Configuration file does not exist at the provided path"
        )
    config = configparser.ConfigParser()
    with open(filename) as confFile:
        config.read_string(confFile.read())
    return config


# Add/update (?) tables in database
# (I'm not actually sure this updates if already existing)


def update_tables(db_url: str) -> None:
    engine = sql.create_engine("sqlite:///" + db_url, echo=False)
    Base.metadata.create_all(engine, checkfirst=True)


# Function for parsing table into list of dicts
def table_parse(table_filepath: str) -> list:
    """Parse table into list of dicts.

    Takes the manually curated metadata table as input and
    turns it into a list of dicts, one entry for each srr with
    key: value pairs for each column in the metadata table
    Output: List of dicts
    """
    # Check that the table file exists
    if not (os.path.exists(table_filepath) and os.path.isfile(table_filepath)):
        raise FileNotFoundError(f"{table_filepath} does not exist.")

    # Load in file as a list of dicts
    table_list = []
    with open(table_filepath, newline="") as tab:
        full_table = csv.DictReader(tab, delimiter="\t")
        for entry in full_table:
            table_list.append(dict(entry))

    return table_list


# Function for grabbing specific keys
def key_grab(table_list, key_list) -> list:
    """Grab specific key values.

    Takes list of dicts and a list of keys and
    extracts specific values to a list for inputting into database
    Output: List of values corresponding to input keys for each
    table entry
    """
    # Load in file as a list of dicts
    value_list = []
    for entry in table_list:
        value_subset = []
        for i in range(len(key_list)):
            value_subset.append(entry[key_list[i]])
        value_list.append(value_subset)

    return value_list


def get_unique_table(location_key, column_keys) -> dict:
    filepath = config["file_locations"][location_key]
    full_table_dict = table_parse(filepath)

    full_table_list = np.array(key_grab(full_table_dict, column_keys))
    unique_list = np.unique(full_table_list, axis=0)

    unique_table = []
    for i in range(len(unique_list)):
        entry = dict(zip(column_keys, unique_list[i]))
        unique_table.append(entry)

    return unique_table


def value_compare(db_row, metatable_row, key_dict) -> bool:
    for key in key_dict:
        if db_row[key] == metatable_row[key_dict[key]]:
            continue
        else:
            return 0
    return 1


def object_as_dict(obj):
    return {c.key: getattr(obj, c.key) for c
            in sql.inspect(obj).mapper.column_attrs}


#
# utils.py ends here

In [20]:
# config = utils.load_config("/home/lsanford/Documents/data/repositories/dbnascent_build/config.txt")
config = load_config(
    "/home/lsanford/Documents/data/repositories/DBNascent-build/config.txt"
)
db_url = config["file_locations"]["database"]

In [21]:
# %load orm.py
# orm.py --- ORM for DBNascent
#
# Filename: orm.py
# Description: ORM for DBNascent
# Authors: Zach Maas and Lynn Sanford
# Maintainer: Lynn Sanford <lynn.sanford@colorado.edu>
# Created: Mon Jun 10 13:11:55 2019 (-0600)
# URL:
#

# Commentary:
#
# This file contains code for an ORM to interface with the Dowell
# Lab's Nascent Database.
#

# Code:

import sqlalchemy as sql
from sqlalchemy.ext.declarative import declarative_base

# Base class for our ORM
Base = declarative_base()


# MAIN TABLES
class organismInfo(Base):
    __tablename__ = "organismInfo"
    organism = sql.Column(
        sql.String(length=127), primary_key=True, index=True, unique=True
    )
    genome_build = sql.Column(sql.String(length=50))
    genome_bases = sql.Column(sql.Integer)

#    def __repr__(self):
#        return f"organismInfo(genome_build={self.genome_build!r}, genome_bases={self.genome_bases!r})"


class searchEq(Base):
    __tablename__ = "searchEq"
    search_term = sql.Column(
        sql.String(length=250), primary_key=True, index=True, unique=True
    )
    db_term = sql.Column(sql.String(length=127))


class exptMetadata(Base):
    __tablename__ = "exptMetadata"
    expt_id = sql.Column(sql.Integer,
                         primary_key=True,
                         index=True,
                         unique=True)
    srp = sql.Column(sql.String(length=50))
    protocol = sql.Column(sql.String(length=50))
    organism = sql.Column(
        sql.String(length=127), sql.ForeignKey("organismInfo.organism")
    )
    library = sql.Column(sql.String(length=50))
    spikein = sql.Column(sql.String(length=127))
    paper_id = sql.Column(sql.String(length=127))
    published = sql.Column(sql.Boolean)
    year = sql.Column(sql.Integer)
    first_author = sql.Column(sql.String(length=127))
    last_author = sql.Column(sql.String(length=127))
    doi = sql.Column(sql.String(length=300))
    curator1 = sql.Column(sql.String(length=50))
    curator2 = sql.Column(sql.String(length=50))
    other_sr_data = sql.Column(sql.Boolean)
    atac_seq = sql.Column(sql.Boolean)
    rna_seq = sql.Column(sql.Boolean)
    chip_seq = sql.Column(sql.Boolean)
    three_dim_seq = sql.Column(sql.Boolean)
    other_seq = sql.Column(sql.Boolean)
    paper_qc_score = sql.Column(sql.Float)
    paper_data_score = sql.Column(sql.Float)


class sampleID(Base):
    __tablename__ = "sampleID"
    srr = sql.Column(sql.String(length=50),
                     primary_key=True,
                     index=True,
                     unique=True)
    sample_name = sql.Column(sql.String(length=50))
    sample_id = sql.Column(sql.Integer)


class geneticInfo(Base):
    __tablename__ = "geneticInfo"
    genetic_id = sql.Column(sql.Integer,
                            primary_key=True,
                            index=True,
                            unique=True)
    organism = sql.Column(
        sql.String(length=127), sql.ForeignKey("organismInfo.organism")
    )
    sample_type = sql.Column(sql.String(length=127))
    cell_type = sql.Column(sql.String(length=127))
    clone_individual = sql.Column(sql.String(length=127))
    strain = sql.Column(sql.String(length=127))
    genotype = sql.Column(sql.String(length=127))
    construct = sql.Column(sql.String(length=127))


class conditionInfo(Base):
    __tablename__ = "conditionInfo"
    condition_id = sql.Column(sql.Integer,
                              primary_key=True,
                              index=True,
                              unique=True)
    condition_type = sql.Column(sql.String(length=127))
    treatment = sql.Column(sql.String(length=127))
    conc_intens = sql.Column(sql.String(length=50))
    start_time = sql.Column(sql.Integer)
    end_time = sql.Column(sql.Integer)
    duration = sql.Column(sql.Integer)
    time_unit = sql.Column(sql.String(length=50))
    duration_unit = sql.Column(sql.String(length=50))


exptCondition = sql.Table(
    "exptCondition",
    Base.metadata,
    sql.Column("sample_id",
               sql.Integer,
               sql.ForeignKey("sampleID.sample_id")),
    sql.Column("condition_id",
               sql.Integer,
               sql.ForeignKey("conditionInfo.condition_id")),
)


class linkIDs(Base):
    __tablename__ = "linkIDs"
    sample_id = sql.Column(
        sql.Integer,
        sql.ForeignKey("sampleID.sample_id"),
        primary_key=True,
        index=True,
        unique=True,
    )
    genetic_id = sql.Column(sql.Integer,
                            sql.ForeignKey("geneticInfo.genetic_id"))
    expt_id = sql.Column(sql.Integer,
                         sql.ForeignKey("exptMetadata.expt_id"))


class sampleAccum(Base):
    __tablename__ = "sampleAccum"
    sample_id = sql.Column(
        sql.Integer,
        sql.ForeignKey("sampleID.sample_id"),
        primary_key=True,
        index=True,
        unique=True,
    )
    replicate = sql.Column(sql.Integer)
    single_paired = sql.Column(sql.String(length=50))
    rcomp = sql.Column(sql.Boolean)
    expt_unusable = sql.Column(sql.Boolean)
    timecourse = sql.Column(sql.Boolean)
    baseline_control_expt = sql.Column(sql.String(length=50))
    notes = sql.Column(sql.String(length=300))
    raw_read_depth = sql.Column(sql.Integer)
    trim_read_depth = sql.Column(sql.Integer)
    raw_read_length = sql.Column(sql.Integer)
    duplication_picard = sql.Column(sql.Float)
    single_map = sql.Column(sql.Integer)
    multi_map = sql.Column(sql.Integer)
    map_prop = sql.Column(sql.Float)
    rseqc_tags = sql.Column(sql.Integer)
    rseqc_cds = sql.Column(sql.Integer)
    rseqc_five_utr = sql.Column(sql.Integer)
    rseqc_three_utr = sql.Column(sql.Integer)
    rseqc_intron = sql.Column(sql.Integer)
    cds_rpk = sql.Column(sql.Float)
    intron_rpk = sql.Column(sql.Float)
    exint_ratio = sql.Column(sql.Float)
    distinct_tenmillion_prop = sql.Column(sql.Float)
    genome_prop_cov = sql.Column(sql.Float)
    avg_fold_cov = sql.Column(sql.Float)
    samp_qc_score = sql.Column(sql.Integer)
    samp_data_score = sql.Column(sql.Integer)


class nascentflowMetadata(Base):
    __tablename__ = "nascentflowMetadata"
    nascentflow_version_id = sql.Column(
        sql.Integer, primary_key=True, index=True, unique=True
    )
    sample_id = sql.Column(sql.Integer, sql.ForeignKey("sampleID.sample_id"))
    nascentflow_version = sql.Column(sql.String(length=127))
    pipeline_revision_hash = sql.Column(sql.String(length=127))
    pipeline_hash = sql.Column(sql.String(length=127))
    nascentflow_date = sql.Column(sql.Date)
    nascentflow_redo_date = sql.Column(sql.Date)
    nextflow_version = sql.Column(sql.String(length=127))
    fastqc_version = sql.Column(sql.String(length=127))
    bbmap_version = sql.Column(sql.String(length=127))
    hisat2_version = sql.Column(sql.String(length=127))
    samtools_version = sql.Column(sql.String(length=127))
    sratools_version = sql.Column(sql.String(length=127))
    preseq_version = sql.Column(sql.String(length=127))
    preseq_date = sql.Column(sql.Date)
    rseqc_version = sql.Column(sql.String(length=127))
    rseqc_date = sql.Column(sql.Date)
    java_version = sql.Column(sql.String(length=127))
    picard_gc_version = sql.Column(sql.String(length=127))
    picard_dups_version = sql.Column(sql.String(length=127))
    picard_date = sql.Column(sql.Date)
    bedtools_version = sql.Column(sql.String(length=127))
    igvtools_version = sql.Column(sql.String(length=127))
    seqkit_version = sql.Column(sql.String(length=127))
    mpich_version = sql.Column(sql.String(length=127))
    gcc_version = sql.Column(sql.String(length=127))
    python_version = sql.Column(sql.String(length=127))
    numpy_version = sql.Column(sql.String(length=127))


class bidirflowMetadata(Base):
    __tablename__ = "bidirflowMetadata"
    bidirflow_version_id = sql.Column(
        sql.Integer, primary_key=True, index=True, unique=True
    )
    sample_id = sql.Column(sql.Integer, sql.ForeignKey("sampleID.sample_id"))
    bidirflow_version = sql.Column(sql.String(length=127))
    pipeline_revision_hash = sql.Column(sql.String(length=127))
    pipeline_hash = sql.Column(sql.String(length=127))
    bidirflow_date = sql.Column(sql.Date)
    nextflow_version = sql.Column(sql.String(length=127))
    samtools_version = sql.Column(sql.String(length=127))
    bedtools_version = sql.Column(sql.String(length=127))
    mpich_version = sql.Column(sql.String(length=127))
    openmpi_version = sql.Column(sql.String(length=127))
    gcc_version = sql.Column(sql.String(length=127))
    r_version = sql.Column(sql.String(length=127))
    rsubread_version = sql.Column(sql.String(length=127))
    boost_version = sql.Column(sql.String(length=127))
    fstitch_version = sql.Column(sql.String(length=127))
    tfit_version = sql.Column(sql.String(length=127))
    dreg_version = sql.Column(sql.String(length=127))
    dreg_date = sql.Column(sql.Date)
    tfit_date = sql.Column(sql.Date)
    fcgene_date = sql.Column(sql.Date)


# The following were created by Zach and we may or may not use...

# class tf(Base):
#    __tablename__ = "tf"
#    tf_id = sql.Column(sql.String(length=127), primary_key=True)
#    tf_alias = sql.Column(sql.String(length=127))


# class pipeline_status(Base):
#    __tablename__ = "pipeline_status"
#    srr_id = sql.Column(
#        sql.String(length=127),
#        sql.ForeignKey("srr_metadata.srr_id"),
#        primary_key=True,
#    )
#    fastqc_complete = sql.Column(sql.Boolean)
#    bbduk_complete = sql.Column(sql.Boolean)
#    hisat2_complete = sql.Column(sql.Boolean)
#    samtools_complete = sql.Column(sql.Boolean)
#    fastq_dump_complete = sql.Column(sql.Boolean)
#    pileup_complete = sql.Column(sql.String(length=127))
#    preseq_complete = sql.Column(sql.Boolean)
#    rseqc_complete = sql.Column(sql.String(length=127))
#    bedtools_complete = sql.Column(sql.Boolean)
#    igv_tools_complete = sql.Column(sql.Boolean)
#    fstitch_complete = sql.Column(sql.Boolean)
#    tfit_complete = sql.Column(sql.Boolean)


# class md_score(Base):
#    __tablename__ = "md_score"
#    srr_id = sql.Column(
#        sql.String(length=127),
#        sql.ForeignKey("srr_metadata.srr_id"),
#        primary_key=True,
#    )
#    tf_id = sql.Column(sql.String, sql.ForeignKey("tf.tf_id"))
#    erna_type = sql.Column(sql.String(length=127))
#    md_score_expected = sql.Column(sql.Integer)
#    md_score_std = sql.Column(sql.Integer)


# orm.py ends here

In [22]:
query_join_keys = dict(config["query_join keys"])

# Find all table columns and make a dict

db_query_names = {}
db_filter_names = {}
tables_to_join = []
db_tables = dict(Base.metadata.tables)
for key in db_tables:
    for column in user_input_fields:
        if column in db_tables[key].columns.keys():
            if column not in db_query_names.keys():
                db_query_names[column] = (key + "." + column)
            if key not in tables_to_join:
                if key not in query_join_keys["existing"]:
                    tables_to_join.append(key)
    for filtkey in user_input_filters:
        if filtkey in db_tables[key].columns.keys():
            if filtkey not in db_filter_names.keys():
                db_filter_names[filtkey] = (key + "." + filtkey)
            if key not in tables_to_join:
                if key not in query_join_keys["existing"]:
                    tables_to_join.append(key)

In [23]:
query_build = "select distinct linkIDs.sample_id, exptMetadata.paper_id, sampleID.sample_name"
for key in db_query_names:
    query_build = query_build + ", " + db_query_names[key]

query_build = (query_build + " from linkIDs " +
               " inner join exptMetadata on exptMetadata.expt_id = linkIDs.expt_id" +
               " inner join sampleID on sampleID.sample_id = linkIDs.sample_id")

for tab in tables_to_join:
    query_build = (query_build + " " + query_join_keys[tab.lower()])

i=0
for key in user_input_filters.keys():
    for value in user_input_filters[key]:
        if i == 0:
            query_build = (query_build + " where " + db_filter_names[key] + " " + value)
            i = 1
        else:
            query_build = (query_build + " and " + db_filter_names[key] + " " + value)

In [24]:
results = []
# with utils.NascentDBConnection(db_url=db_url) as session:
with NascentDBConnection(db_url="sqlite:///" + db_url) as session:
    results = session.execute(query_build).fetchall()

In [25]:
with open(outfile, "w") as f:
    w = csv.writer(f)
    w.writerow((["sample_id", "paper_id", "sample_name"] + list(db_query_names.keys())))
    for data in results:
        w.writerow(data)